In [2]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [3]:
# X = np.random.uniform(low=-2, high=5, size=(6,8))
# ytrue = np.array([1,0,1,2,0,2])

def get_data(path):
    data = pd.read_csv(path, index_col=0)

    cols = list(data.columns)
    target = cols.pop()

    X = data[cols].copy()
    y = data[target].copy()

    y = LabelEncoder().fit_transform(y)

    return np.array(X), np.array(y)

X, y = get_data(r'C:\Users\12482\Desktop\articles\nn_from_scratch\iris.csv')

In [47]:
class DenseLayer:
    def __init__(self, neurons):
        self.neurons = neurons
        
    def relu(self, inputs):
        return np.maximum(0, inputs)

    def softmax(self, inputs):
        exp_scores = np.exp(inputs)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        return probs
    
    def relu_derivative(self, dA, Z):
        dZ = np.array(dA, copy = True)
        dZ[Z <= 0] = 0
        return dZ
    
    def forward(self, inputs, weights, bias, activation):
        Z_curr = np.dot(inputs, weights.T) + bias
        
        if activation == 'relu':
            A_curr = self.relu(inputs=Z_curr)
        elif activation == 'softmax':
            A_curr = self.softmax(inputs=Z_curr)
            
        return A_curr, Z_curr
        
class Network:
    def __init__(self):
        self.network = [] ## layers
        self.architecture = [] ## mapping input neurons --> output neurons
        self.params = [] ## W, b
        self.memory = [] ## Z, A
        
    def add(self, layer):
        self.network.append(layer)
            
    def _compile(self, data):
        for idx, layer in enumerate(self.network):
            if idx == 0:
                self.architecture.append({'input_dim':data.shape[1], 
                                          'output_dim':layer.neurons, 'activation':'relu'})
            if idx == len(self.network)-2:
                self.architecture.append({'input_dim':layer.neurons, 
                                          'output_dim':self.network[idx+1].neurons, 'activation':'softmax'})
            elif idx != len(self.network)-2 and idx != len(self.network)-1:
                self.architecture.append({'input_dim':layer.neurons, 
                                          'output_dim':self.network[idx+1].neurons, 'activation':'relu'})
            else:
                continue
                
        return self
    
    def _init_weights(self, data):
        self._compile(data)
        
        for i in range(len(self.architecture)):
            self.params.append({
                'W':np.random.uniform(low=-1, high=1, 
                  size=(self.architecture[i]['output_dim'], 
                        self.architecture[i]['input_dim'])),
                'b':np.zeros((1, self.architecture[i]['output_dim']))})
        
        return self
    
    def _forwardprop(self, data):
        self._init_weights(data)
        
        A_curr = data
        
        for i in range(len(self.params)):
            A_prev = A_curr
            A_curr, Z_curr = self.network[i].forward(inputs=A_prev, weights=self.params[i]['W'], 
                                           bias=self.params[i]['b'], activation=self.architecture[i]['activation'])
            
            self.memory.append({'inputs':A_prev, 'Z':Z_curr})
            
        return A_curr
    
    def _get_accuracy(self, predicted, actual):
        return np.mean(np.argmax(predicted, axis=1)==actual)
    
    def _calculate_loss(self, predicted, actual):
        samples = len(actual)

        out_clipped = np.clip(predicted, 1e-7, 1-1e-7)

        if len(actual.shape) == 1:
            confs = out_clipped[range(samples), actual]
        elif len(actual.shape) == 2:
            confs = np.sum(out_clipped*actual, axis=1)

        return np.mean(-np.log(confs))
        

model = Network()
model.add(DenseLayer(X.shape[0]))
model.add(DenseLayer(8))
model.add(DenseLayer(10))
model.add(DenseLayer(3))

yhat = model._forwardprop(X)
accuracy = model._get_accuracy(predicted=yhat, actual=y)
loss = model._calculate_loss(predicted=yhat, actual=y)

assert(len(yhat)==len(y))

# print('PREDICTED:')
# print(np.argmax(yhat, axis=1))

# print()

# print('ACTUAL:')
# print(y)

# print()

# print('ACCURACY:', accuracy)
# print('LOSS:', loss)

In [48]:
loss

6.146390893880092

In [49]:
probs = yhat / np.sum(yhat, axis=1, keepdims=True)
correct_logprobs = -np.log(probs[range(150),y])
data_loss = np.sum(correct_logprobs)/150

data_loss

6.146390893880092